In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Constants
VOCAB_SIZE = 10000
MAXLEN = 500
BATCH_SIZE = 64

print("Loading data...")
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=MAXLEN)
input_test = sequence.pad_sequences(input_test, maxlen=MAXLEN)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

def build_model(rnn_cell):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 32),
        rnn_cell(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

rnn_model = build_model(tf.keras.layers.SimpleRNN)

lstm_model = build_model(tf.keras.layers.LSTM)

gru_model = build_model(tf.keras.layers.GRU)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history = rnn_model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[callback])

results = rnn_model.evaluate(input_test, y_test)
print(f'Test Loss: {results[0]} - Test Accuracy: {results[1]}')


Loading data...
17464789/17464789 [==============================] - 82s 5us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)
Epoch 1/10
313/313 [==============================] - 147s 466ms/step - loss: 0.6757 - accuracy: 0.5760 - val_loss: 0.6410 - val_accuracy: 0.6322
Epoch 2/10
313/313 [==============================] - 144s 459ms/step - loss: 0.6123 - accuracy: 0.6758 - val_loss: 0.6122 - val_accuracy: 0.6470
Epoch 3/10
313/313 [==============================] - 139s 444ms/step - loss: 0.4953 - accuracy: 0.7710 - val_loss: 0.5729 - val_accuracy: 0.7168
Epoch 4/10
313/313 [==============================] - 144s 460ms/step - loss: 0.3532 - accuracy: 0.8449 - val_loss: 0.4629 - val_accuracy: 0.7960
Epoch 5/10
313/313 [==============================] - 146s 468ms/step - loss: 0.2465 - accuracy: 0.9025 - val_loss: 0.4792 - val_accuracy: 0.7922
Epoch 6/10
782/782 [==============================]

In [2]:
history_LSTM = lstm_model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[callback])

results_LSTM = lstm_model.evaluate(input_test, y_test)
print(f'Test Loss: {results_LSTM[0]} - Test Accuracy: {results_LSTM[1]}')

Epoch 1/10
313/313 [==============================] - 8s 19ms/step - loss: 0.4904 - accuracy: 0.7605 - val_loss: 0.3512 - val_accuracy: 0.8536
Epoch 2/10
313/313 [==============================] - 5s 17ms/step - loss: 0.2600 - accuracy: 0.8992 - val_loss: 0.2961 - val_accuracy: 0.8786
Epoch 3/10
313/313 [==============================] - 5s 18ms/step - loss: 0.1923 - accuracy: 0.9327 - val_loss: 0.3361 - val_accuracy: 0.8524
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3502 - accuracy: 0.8548
Test Loss: 0.3501717448234558 - Test Accuracy: 0.8547599911689758


In [3]:
history_GRU = gru_model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[callback])

results_GRU = gru_model.evaluate(input_test, y_test)
print(f'Test Loss: {results_GRU[0]} - Test Accuracy: {results_GRU[1]}')

Epoch 1/10
313/313 [==============================] - 7s 19ms/step - loss: 0.4882 - accuracy: 0.7487 - val_loss: 0.3499 - val_accuracy: 0.8558
Epoch 2/10
313/313 [==============================] - 5s 17ms/step - loss: 0.2601 - accuracy: 0.8963 - val_loss: 0.3019 - val_accuracy: 0.8746
Epoch 3/10
313/313 [==============================] - 5s 17ms/step - loss: 0.1848 - accuracy: 0.9317 - val_loss: 0.3161 - val_accuracy: 0.8792
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3708 - accuracy: 0.8645
Test Loss: 0.37075895071029663 - Test Accuracy: 0.8645200133323669


In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten
from tensorflow.keras.models import Model

VOCAB_SIZE = 10000 
MAXLEN = 500  

def build_model2(rnn_cell, rnn_units=32):
    sequence_input = Input(shape=(MAXLEN,), dtype='int32')
    embedded_sequences = Embedding(VOCAB_SIZE, rnn_units)(sequence_input)
    rnn_output = rnn_cell(rnn_units, return_sequences=True)(embedded_sequences)
    
    query_value_attention_seq = tf.keras.layers.Attention()([rnn_output, rnn_output])
    
    attention_flatten = Flatten()(query_value_attention_seq)

    dense_output = Dense(1, activation='sigmoid')(attention_flatten)
    
    model = Model(inputs=sequence_input, outputs=dense_output)
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

# LSTM Model with Attention
lstm_model_with_attention = build_model2(tf.keras.layers.LSTM)
lstm_model_with_attention.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 500, 32)      320000      ['input_5[0][0]']                
                                                                                                  
 lstm_7 (LSTM)                  (None, 500, 32)      8320        ['embedding_9[0][0]']            
                                                                                                  
 attention_6 (Attention)        (None, 500, 32)      0           ['lstm_7[0][0]',                 
                                                                  'lstm_7[0][0]']           

In [14]:

history_ = lstm_model_with_attention.fit(input_train, y_train,
                    epochs=10,
                    batch_size=BATCH_SIZE,
                    validation_split=0.2,
                    callbacks=[callback])

results_ = lstm_model_with_attention.evaluate(input_test, y_test)
print(f'Test Loss: {results_[0]} - Test Accuracy: {results_[1]}')

Epoch 1/10


313/313 [==============================] - 10s 27ms/step - loss: 0.4309 - accuracy: 0.7860 - val_loss: 0.3505 - val_accuracy: 0.8540
Epoch 2/10
313/313 [==============================] - 8s 25ms/step - loss: 0.2237 - accuracy: 0.9115 - val_loss: 0.3671 - val_accuracy: 0.8456
Epoch 3/10
313/313 [==============================] - 8s 25ms/step - loss: 0.1579 - accuracy: 0.9392 - val_loss: 0.3504 - val_accuracy: 0.8838
Epoch 4/10
313/313 [==============================] - 8s 25ms/step - loss: 0.1078 - accuracy: 0.9582 - val_loss: 0.4100 - val_accuracy: 0.8732
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.6136 - accuracy: 0.8587
Test Loss: 0.6136218309402466 - Test Accuracy: 0.8587200045585632
